In [1]:
from dataclasses import dataclass
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from functions import *

from tqdm import tqdm

In [8]:
@dataclass
class Config:
    nx: int = 60
    ny: int = 24
    steps: int = 1000
    output_stride: int = 50
    molecular_weights: tuple[float, float, float] = (28.0, 2.0, 44.0)  # N2, H2, CO2
    # Left half: 50% N2 + 50% H2, Right half: 50% N2 + 50% CO2
    #left_frac: tuple[float, float, float] = (0.5, 0.5, 0.0)
    #right_frac: tuple[float, float, float] = (0.5, 0.0, 0.5)
    left_frac: tuple[float, float, float] = (0.5, 0.25, 0.35)
    right_frac: tuple[float, float, float] = (0.5, 0.35, 0.25)
    total_pressure: float = 1.0
    theta: float = THETA
    nB: int = 5
    frames_dir: str = "demo_frames/three_species_mixing"

In [9]:
def initialise_chamber(config: Config):
    nx, ny = config.nx, config.ny
    species = 3
    phi = 1.0 / np.array(config.molecular_weights, dtype=np.float64)

    # Partial pressures for each species
    psigma = np.zeros((species, nx, ny), dtype=np.float64)
    mid = nx // 2
    left = np.array(config.left_frac, dtype=np.float64) * config.total_pressure
    right = np.array(config.right_frac, dtype=np.float64) * config.total_pressure
    for s in range(species):
        psigma[s, :mid, :] = left[s]
        psigma[s, mid:, :] = right[s]

    # Convert partial pressures to densities: p_s = phi_s * rho_s / 3
    rho_s = np.zeros_like(psigma)
    for s in range(species):
        rho_s[s] = 3.0 * psigma[s] / phi[s]

    # Initial populations from equilibrium with zero velocity
    f = np.zeros((species, 9, nx, ny), dtype=np.float64)
    ux_s = np.zeros((species, nx, ny), dtype=np.float64)
    uy_s = np.zeros((species, nx, ny), dtype=np.float64)
    feq = equilibrium(f, rho_s, phi, ux_s, uy_s)
    f[...] = feq
    return f, phi

In [10]:
def save_concentration_frames(f: np.ndarray, phi: np.ndarray, frame_idx: int, out_dir: Path, molecular_weights) -> None:
    rho_s, _, _, rho_mix, _ = calculate_moment(f, phi)
    rho_mix_safe = np.where(rho_mix > 0.0, rho_mix, 1.0)
    conc = rho_s #/ rho_mix_safe[None, :, :]

    labels = ["N2", "H2", "CO2"]
    cmaps = ["Blues", "Greens", "Reds"]

    fig, axes = plt.subplots(3, 1, figsize=(6, 9), sharex=True, sharey=True)
    for s in range(3):
        im = axes[s].imshow((conc[s].T)/molecular_weights[s], origin="lower", cmap=cmaps[s], aspect="auto")
        axes[s].set_title(f"{labels[s]} concentration")
        axes[s].set_xlabel("x")
        axes[s].set_ylabel("y")
        fig.colorbar(im, ax=axes[s], fraction=0.046, pad=0.04)
    fig.tight_layout()
    fig.savefig(f"demo_frames_triple/frame_{frame_idx:04d}.png", dpi=160)
    plt.close(fig)

In [11]:
def main():
    cfg = Config()
    out_dir = 'demo_frames'

    f, phi = initialise_chamber(cfg)
    molecular_weights = np.array(cfg.molecular_weights, dtype=np.float64)

    for step in tqdm(range(cfg.steps + 1)):
        if step % cfg.output_stride == 0:
            save_concentration_frames(f, phi, step, out_dir, molecular_weights)
        if step == cfg.steps:
            break
        #f = bgk_step(f, phi, molecular_weights, nB=cfg.nB, theta=cfg.theta)
        f = bgk_step(f, molecular_weights, phi, cfg.nB, lattice_stream_bounce_xy)

    print(f"Saved frames to {out_dir}")

In [12]:
main()

100%|█████████▉| 1000/1001 [00:16<00:00, 60.08it/s]

Saved frames to demo_frames
